In [17]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import logging
from typing import Dict, List, Tuple, Any
import mlflow
from datetime import datetime

In [18]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('erp_bot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [19]:
class CBv2Dataset(Dataset):
    def __init__(self, texts: List[str], main_intents: List[str], sub_intents: List[str], tokenizer: Any, max_len: int = 128):
        self.texts = texts
        self.main_intents = main_intents
        self.sub_intents = sub_intents
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        main_intent = self.main_intents[idx]
        sub_intent = self.sub_intents[idx]

        encoding = self.tokenizer.encode_plus(
            text, 
            add_special_tokens = True,
            max_length = self.max_len,
            padding = 'max_length',
            truncation = True,
            return_attention_mask = True,
            return_tensors = 'pt'
        )

        return {
            'text' : text,
            'input_ids' : encoding['input_ids'].flatten(),
            'attention_mask' : encoding['attention_mask'].flatten(),
            'main_intent' : torch.tensor(main_intent, dtype = torch.long),
            'sub_intent' : torch.tensor(sub_intent, dtype = torch.long)
        }

In [20]:
class HierarchicalIntentClassifier(nn.Module):
    def __init__(self, intent_structure: Dict[str, Dict[str, List[str]]]):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(0.3)
        
        # Main intent classifier
        self.n_main_intents = len(intent_structure)
        self.main_classifier = nn.Linear(768, self.n_main_intents)
        
        # Create sub-classifiers for each main intent
        self.sub_classifiers = nn.ModuleDict({
            main_intent: nn.Linear(768, len(sub_intents))
            for main_intent, sub_intents in intent_structure.items()
        })
        
        # Store the structure for reference
        self.intent_structure = intent_structure
        
    def forward(self, input_ids, attention_mask, main_intent=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropped = self.drop(pooled_output)
        
        # Get main intent prediction
        main_output = self.main_classifier(dropped)
        
        if main_intent is None:
            # During prediction, use predicted main intent
            main_intent = torch.argmax(main_output, dim=1)
        
        # Get sub-intent prediction for the specific main intent
        batch_size = input_ids.size(0)
        sub_outputs = torch.zeros(batch_size, max(len(subs) for subs in self.intent_structure.values()))
        sub_outputs = sub_outputs.to(input_ids.device)
        
        for i in range(batch_size):
            intent_name = list(self.intent_structure.keys())[main_intent[i]]
            sub_classifier = self.sub_classifiers[intent_name]
            sub_output = sub_classifier(dropped[i].unsqueeze(0))
            # Pad if necessary
            sub_outputs[i, :sub_output.size(1)] = sub_output
            
        return main_output, sub_outputs

In [22]:
class IntentClassifier:
    def __init__(self, model_path: str = None, confidence_threshold: float = 0.7):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = None
        self.intent_structure = {}
        self.main_intent_to_id = {}
        self.sub_intent_to_id = {}
        self.confidence_threshold = confidence_threshold
        
        if model_path:
            self.load_model(model_path)

    def load_model(self, path: str):
        checkpoint = torch.load(path, map_location=self.device)
        self.intent_structure = checkpoint['intent_structure']
        self.model = HierarchicalIntentClassifier(self.intent_structure)
        self.model.load_state_dict(checkpoint['model_state'])
        self.main_intent_to_id = checkpoint['main_intent_to_id']
        self.sub_intent_to_id = checkpoint['sub_intent_to_id']
        self.model.to(self.device)
        logger.info(f"Model loaded from {path}")

    def predict(self, text: str) -> Tuple[str, str, float, float]:
        self.model.eval()
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)
        
        with torch.no_grad():
            main_outputs, _ = self.model(input_ids, attention_mask)
            main_probs = torch.softmax(main_outputs, dim=1)
            main_confidence, main_predicted = torch.max(main_probs, 1)
            
            # Check confidence threshold
            if main_confidence.item() < self.confidence_threshold:
                return (
                    "out_of_context_contact_llm",
                    "handle_unknown_query",
                    main_confidence.item(),
                    0.0
                )
            
            # If confidence is good, proceed with normal prediction
            main_id_to_intent = {v: k for k, v in self.main_intent_to_id.items()}
            predicted_main_intent = main_id_to_intent[main_predicted.item()]
            
            # Get sub-intent prediction
            dropped = self.model.drop(self.model.bert(input_ids, attention_mask)[1])
            sub_classifier = self.model.sub_classifiers[predicted_main_intent]
            sub_outputs = sub_classifier(dropped)
            sub_probs = torch.softmax(sub_outputs, dim=1)
            sub_confidence, sub_predicted = torch.max(sub_probs, 1)
            
            sub_id_to_intent = {v: k for k, v in self.sub_intent_to_id[predicted_main_intent].items()}
            predicted_sub_intent = sub_id_to_intent[sub_predicted.item()]
        
        return (
            predicted_main_intent,
            predicted_sub_intent,
            main_confidence.item(),
            sub_confidence.item()
        )


In [23]:
def handle_user_query(text: str, classifier: IntentClassifier):
    main_intent, sub_intent, main_conf, sub_conf = classifier.predict(text)
    
    if main_intent == "out_of_context_contact_llm":
        logger.info(f"Query '{text}' detected as out-of-context (confidence: {main_conf:.2f})")
        logger.info("Forwarding to LLM microservice")
        return {
            'status': 'llm_fallback',
            'confidence': main_conf,
            'original_query': text,
            #Add your LLM microservice call here
            'action': 'forward_to_llm'
        }
    else:
        logger.info(f"Query '{text}' matched intent: {main_intent}/{sub_intent}")
        logger.info(f"Confidence scores - Main: {main_conf:.2f}, Sub: {sub_conf:.2f}")
        return {
            'status': 'intent_matched',
            'main_intent': main_intent,
            'sub_intent': sub_intent,
            'confidence': {
                'main': main_conf,
                'sub': sub_conf
            }
        }


In [32]:
classifier = IntentClassifier(confidence_threshold=0.8)
classifier.load_model('models/optimized_model.pt')

2025-05-09 10:38:20,065 - __main__ - INFO - Model loaded from models/optimized_model.pt


In [41]:
test_messages = [
    "How many working days are there in a week?", 
]


In [42]:
for message in test_messages:
    result = handle_user_query(message, classifier)
    print(f"\nQuery: {message}")
    print(f"Result: {result}")

2025-05-09 10:39:49,898 - __main__ - INFO - Query 'How many working days are there in a week?' matched intent: employee_module/shift_master
2025-05-09 10:39:49,898 - __main__ - INFO - Confidence scores - Main: 0.94, Sub: 0.25



Query: How many working days are there in a week?
Result: {'status': 'intent_matched', 'main_intent': 'employee_module', 'sub_intent': 'shift_master', 'confidence': {'main': 0.9369529485702515, 'sub': 0.24527935683727264}}
